<h1><u>Grouping & Aggregating Data</u></h1>


### 1. Average Number of Orders per User
### 2. Creating a Loyalty Flag for customers using transform & loc
### 3. Loyalty Flag Statistics
### 4. Classifying Spenders
### 5. Classifying frequent customers

In [3]:
## importing libraries & data 
import pandas as pd 
import numpy as np 
import os 

path = r'/Users/aahamoustafa/Desktop/Data Projects/Instacart Basket Analysis  12-2024'

ords_prods_merge =  pd.read_pickle(os.path.join(path,'02 Data', 'Prepared Data', 'ords_prods_latest.pkl'))


In [46]:
ords_prods_merge.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,department_id,prices,_merge,price_range_loc,Busiest_day,Busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,7,9.0,both,Mid range Product,Regularly Busy,Average Orders,10,New customer,6.367797,Low spender
1,2539329,1,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,...,16,12.5,both,Mid range Product,Regularly Busy,Average Orders,10,New customer,6.367797,Low spender
2,2539329,1,1,2,8,NaN,12427,3,0,Original Beef Jerky,...,19,4.4,both,Low range Product,Regularly Busy,Average Orders,10,New customer,6.367797,Low spender
3,2539329,1,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,...,19,4.7,both,Low range Product,Regularly Busy,Average Orders,10,New customer,6.367797,Low spender
4,2539329,1,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,...,17,1.0,both,Low range Product,Regularly Busy,Average Orders,10,New customer,6.367797,Low spender


In [12]:
ords_prods_merge.shape

(32404859, 17)

## 1. Average Number of Orders Per user in each department 

In [87]:
ords_prods_merge.groupby('department_id')['order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

## 2. Creating a Loyalty Flag For Customers

In [9]:
## First creating a max order column for each user in every row using transform 

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


/var/folders/_6/h3nvtrc54137rkl3n8zpfwnc0000gn/T/ipykernel_4879/3304103766.py:3: FutureWarning: The provided callable <function max at 0x104754400> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [11]:
## creating loyalty flag column
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [13]:
## checking counts of loyalty flag
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

## 3. Loyalty Flag Spending Statistics 

In [32]:
# Calculate total spending per customer
avg_spending_per_customer = ords_prods_merge.groupby(['user_id', 'loyalty_flag'])['prices'].sum().reset_index()


avg_spending_per_customer.sort_values(by = 'prices', ascending = False) 

,user_id,loyalty_flag,prices
201267,201268,Loyal customer,26394.9
129927,129928,Loyal customer,24307.1
164054,164055,Loyal customer,22777.9
137628,137629,Loyal customer,22696.9
176477,176478,Loyal customer,22595.5
...,...,...,...
183949,183950,New customer,3.3
62097,62098,New customer,3.3
55711,55712,New customer,3.3
16697,16698,New customer,3.0


In [23]:
# Now Group by loyalty_flag to compare basic spending statistics 
avg_spending_per_customer.groupby('loyalty_flag')['prices'].describe()


,count,mean,std,min,25%,50%,75%,max
loyalty_flag,,,,,,,,
Loyal customer,17017.0,4697.901347,2643.270763,156.0,2804.800000,4220.90,6058.300,26394.9
New customer,112328.0,433.646473,337.728546,3.0,192.600000,347.50,580.100,3920.4
Regular customer,76864.0,1610.783159,1169.632634,15.7,795.554737,1319.05,2096.425,12186.6


#### We can see that loyal customers are much less in comparison with new and regular customers( but they tend to spend much more on average). With the most loyal customer spending $26394 on his own!

## 4. Targeting different types of spenders 

#### 1. First we will create the new column average_price using transform()

In [38]:
## First creating a new average price column 

ords_prods_merge['average_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform('mean')

In [40]:
## creating a spending flag

ords_prods_merge.loc[ords_prods_merge['average_price'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[(ords_prods_merge['average_price'] >= 10) , 'spending_flag'] = 'High Spender'
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     32285150
High Spender      119709
Name: count, dtype: int64

#### 2. If you just want to classify the 206209 users without creating a new column

In [42]:
user_avg_spending = ords_prods_merge.groupby('user_id')['prices'].mean().reset_index()

user_avg_spending['spending_flag'] = np.where(
    user_avg_spending['prices'] < 10, 'Low Spender', 'High Spender'
) ## alternative way of creating spending flag 

user_spending_flag_counts = user_avg_spending['spending_flag'].value_counts()
print(user_spending_flag_counts)


spending_flag
Low Spender     202821
High Spender      3388
Name: count, dtype: int64


## 5. Finding Frequent vs Non-Frequent Users for Notification Purposes

In [48]:
## First creating a new average price column 

ords_prods_merge['median_days_since_prior_order'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [50]:
## creating a frequency flag
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] <= 10, 'freq_flag'] = 'Frequent Customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since_prior_order'] > 10) & (ords_prods_merge['median_days_since_prior_order'] <= 20) , 'freq_flag'] = 'Normal Customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since_prior_order'] > 20, 'freq_flag'] = 'Non-frequent customer'

ords_prods_merge[['user_id', 'median_days_since_prior_order', 'freq_flag']].head()

,user_id,median_days_since_prior_order,freq_flag
0,1,20.5,Non-frequent customer
1,1,20.5,Non-frequent customer
2,1,20.5,Non-frequent customer
3,1,20.5,Non-frequent customer
4,1,20.5,Non-frequent customer


In [52]:
## exporting the data 

ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_aggregations.pkl'))